# Introduction

julie.tsitron@parks.nyc.gov 1/8/2020

This notebook is for testing various ArcGIS API for Python functions that allow for pushing/updating GIS data to production servers. Ultimately, the script from this notebook can be used as a template for the technical implementation (i.e., pushing clean data into production ESRI-based geodatabases) of Structures, Sites and Units, CPAs, and other Agency Data Model data products.

# Imports and Connections to DBs

In [ ]:
import pandas as pd
import pyodbc
import sys
sys.path.append('../') ## <-- THIS IS THE PART THAT TELLS IT TO LOOK IN THE PARENT DIRECTORY
from IPM_Shared_Code.Python.geo_functions import read_geosql
from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer
# import requests
# from urllib3 import PoolManager
# http = PoolManager()
import json
# import time
from datetime import datetime
import arcgis
import utils
import urllib
import sqlalchemy
import os
import shapely
import smtplib
# from shapely.geometry import shape, mapping

In [ ]:
config = utils.get_config('config.ini')

driver = config['srv']['driver']
vpipm_server = config['srv']['vpipm']
parksgis_dev_server = config['srv']['parksgis_dev']
parksgis_prod_server = config['srv']['parksgis_prod']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']

portal_dev = config['url']['portal_dev']
portal_prod = config['url']['portal_prod']
structures_dev_url = config['url']['structures_dev']
structures_prod_url = config['url']['structures_prod']

In [ ]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' +
                                 vpipm_server + ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [ ]:
connection = engine.connect()

In [ ]:
con = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con_vpipm = pyodbc.connect('Driver={' + driver + '};Server=' + vpipm_server +
                           ';Database=;Trusted_Connection=Yes;')

# Data

# Deal with Dates

## m/d/Y H:M:S format:

In [ ]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev['COMMISSIONDATE']

In [ ]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev.columns.values

## Delta Table from structuresdb (on vpipm)

In [ ]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con_vpipm,
                            geom_raw='shape',
                            geom_col='geometry')

In [ ]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [ ]:
struct_deltas.head()

In [ ]:
# multipolygons

In [ ]:
multipolygons = []
for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
#         print('hi')
        multipolygons.append(row['SYSTEM'])
#         print(i)
# print(row['SYSTEM'])
if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons[:1])
#     print(type(ids))
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

In [ ]:
# def to_wkt(row):
#     return row.wkt

# Write Delta Table to geojson file

In [ ]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\AgencyDataModel\\Develop\\Structures\\delta_snapshots/' + today
if os.path.exists(snapshot):
    struct_deltas.to_file(snapshot + '/deltas.geojson',
                          encoding='utf-8',
                          driver='GeoJSON')
else:
    os.makedirs(snapshot)
    struct_deltas.to_file(snapshot + '/deltas.geojson',
                          encoding='utf-8',
                          driver='GeoJSON')

# Read geojson file to geojson object 

In [ ]:
with open(snapshot+'/deltas.geojson') as data:
    geojson_deltas = json.load(data)

# Create arcgis featureSet from geojson object

In [ ]:
fromJSON_deltas = arcgis.features.FeatureSet.from_geojson(geojson_deltas)

# Connect to published dataset via GIS object

In [ ]:
gis = GIS(url=portal_dev)

# Connect to feature layer directly

In [ ]:
strct_lyr_url = structures_dev_url

In [ ]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [ ]:
structures_features

In [ ]:
len(struct_deltas)

# Make Edits

In [ ]:
crsr = con_vpipm.cursor()
sql_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

In [ ]:
update_result = lyr_structures.edit_features(updates=fromJSON_deltas.features)

In [ ]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()